## https://github.com/BCG-Gamma/facet
. sklearndf
# conda install -c bcg_gamma sklearndf 
# conda install sklearndf -c bcg_gamma -c conda-forge
. gamma-facet
## conda install gamma-facet -c bcg_gamma -c conda-forge



In [43]:
# standard imports
import pandas as pd
import numpy as np 
from sklearn.model_selection import RepeatedKFold

# some helpful imports from sklearndf
from sklearndf.pipeline import PipelineDF, ClassifierPipelineDF
from sklearndf.pipeline import RegressorPipelineDF
# from sklearndf.classification import RandomForestClassifierDF
from sklearndf.regression import RandomForestRegressorDF

# relevant FACET imports
from facet.data import Sample
from facet.selection import LearnerRanker, LearnerGrid
trainDataFile ='./data/TRCT_TrainingData_20210131.csv'
# target_ToolGID = 'PW_PIX'

In [64]:
def preHandleDat(df,isTrain=True):
    from sklearn.preprocessing import StandardScaler #平均&變異數標準化 平均值為0，方差為1。
    from sklearn.preprocessing import MinMaxScaler #最小最大值標準化[0,1]
    from sklearn.preprocessing import RobustScaler #中位數和四分位數標準化
    from sklearn.preprocessing import MaxAbsScaler #絕對值最大標準化
    from pickle import dump
    from pickle import load
    #=================
    #刪除不必要的欄位
    #=================


    # drop_cols=['MFG_DATE','ARRIVAL_WIP_QTY','MOVE_QTY','PROCESS_TIME','C_TC']
    # num_cols=['M_NUM','UP_TIME','C_UP_TIME','LOT_SIZE','C_LOT_SIZE','EQP_UTIL','C_EQP_UTIL','U','HOLD_RATE','ENG_LOT_RATE','HOT_LOT_RATE','QUE_LOT_RATE','NO_HOLD_QTY','WIP_QTY','IS_HOLIDAY','RUN_WIP_RATIO'] 
    drop_cols=['MFG_DATE','ARRIVAL_WIP_QTY','MOVE_QTY']
    num_cols=['M_NUM','UP_TIME','C_UP_TIME','LOT_SIZE','C_LOT_SIZE','EQP_UTIL','C_EQP_UTIL','U','PROCESS_TIME','C_TC','HOLD_RATE','ENG_LOT_RATE','HOT_LOT_RATE','QUE_LOT_RATE','NO_HOLD_QTY','WIP_QTY','IS_HOLIDAY','RUN_WIP_RATIO']

    target_cols=['TRCT']
    cat_cols = ['TOOLG_ID']
    df = df.drop(drop_cols, axis=1)
    #df_train['Finish']=np.where(df_train['TOOLG_ID_NEXT'].isnull(), 1, 0).astype(str)
    #df_train['WEEKDAY']=pd.to_datetime(df_train['MFG_DATE'],format='%Y%m%d').dt.day_name()
    # df['MFG_DATE']=pd.to_datetime(df_train['MFG_DATE'],format='%Y%m%d')
    #'IS_HOLIDAY'
     
    # df = df.drop('WIP_QTY', axis=1)
    # df = df.sort_values('NO_HOLD_QTY')
    # print("DATA Start====")
    # print(df.info())  
    #========================
    # 缺漏值填空
    #========================
    # df = df.fillna(df.median())
    #df = df.fillna(method='bfill') #往後
    df = df.fillna(method='ffill') #往前
    # df = df.fillna(df.mean())  #用平均值取代 nan   
    # df = df.dropna() # 刪除null值   
    #df['ColA'].fillna(value=0, inplace=True) #用 0 取代 nan
    #df['ColA'].fillna(value=df.groupby('ColB')['ColA'].transform('mean'), inplace=True) #利用 groupby()同一group 的平均值

    #==================================================
    #1.特徵縮放
    #==================================================
    
    #特徵縮放欄位 List(排除Target TRCT)==================
    # num_cols=['M_NUM','UP_TIME','C_UP_TIME','LOT_SIZE','C_LOT_SIZE','EQP_UTIL','C_EQP_UTIL','U','PROCESS_TIME','WIP_QTY','NO_HOLD_QTY','MOVE_QTY','ARRIVAL_WIP_QTY','RUN_WIP_RATIO','CLOSE_WIP_QTY','MANAGEMENT_WIP_QTY','C_TC','C_CLOSE_WIP','C_TOOLG_LOADING','C_TOOL_LOADING','DISPATCHING','HOLD_RATE','ENG_LOT_RATE','HOT_LOT_RATE','[BACKUP]','REWORK_RATE','QUE_LOT_RATE','SAMPLING_RATE','NUM_RECIPE','CHANGE_RECIPE','BATCH_SIZE']

   #'ARRIVAL_WIP_QTY',,'WIP_QTY'
    df_train_scal = df.copy(deep=False)
    global df_cols
    df_cols = df_train_scal.columns
    
    if isTrain:
        
        
        #rescaling 特徵縮放 StandardScaler-------------------------------------
        # scaler = StandardScaler()
        # scaler.fit(df_train[num_cols])
        # df_train_scal[num_cols]= scaler.transform(df_train_scal[num_cols])
    
        #rescaling 特徵縮放 MinMaxScaler-------------------------------------
        scaler = MinMaxScaler()
        scaler.fit(df[num_cols])
        df_train_scal[num_cols]= scaler.transform(df_train_scal[num_cols])

        #rescaling 特徵縮放 RobustScaler-------------------------------------
        # scaler = RobustScaler()
        # scaler.fit(df_train[num_cols])
        # df_train_scal[num_cols] = scaler.transform(df_train_scal[num_cols])
        
       
        dump(scaler, open('scaler.pkl', 'wb'))
        # # save the scaler
        # dump(scaler, open('scaler.pkl', 'wb'))
    else:
        # load the scaler
        scaler = load(open('scaler.pkl', 'rb'))
        df_train_scal[num_cols] = scaler.transform(df[num_cols])


    df_train_scal.to_csv('./data/df_train_scal.csv')
    #==================================================
    #2.one hot encoder
    #==================================================
    # target_cols=['MOVE_QTY']
 
    # global df2_train_eh_before
    df_train_eh =pd.get_dummies(df_train_scal.drop(target_cols, axis=1),columns=cat_cols)
    df_train_eh.to_csv('./data/df2_train_eh_before.csv')

    
    
    # # Get missing columns in the training test
    # eh_cols =  getDummyToolG()
    # missing_cols = set( eh_cols ) - set( df_train_eh.columns )

    # # Add a missing column in test set with default value equal to 0
    # for c in missing_cols:
    #     df_train_eh[c] = 0
   
    
    if isTrain:
        df2_train_eh_before = df_train_eh.copy(deep=False)
        df2_train_eh_before.head(0).to_csv('./data/train_eh.csv',index=0) #不保存行索引
    else:
        df2_train_eh_before=readDataFromFile('./data/train_eh.csv')
        df_train_eh = df_train_eh.reindex(columns = df2_train_eh_before.columns, fill_value=0)
        # Ensure the order of column in the test set is in the same order than in train set
        df_train_eh = df_train_eh[df2_train_eh_before.columns]  
 

    df_train_eh.to_csv('./data/df_train_eh.csv')
    # if isTrain:
    #     df2_train_eh_before = df_train_eh.copy(deep=False)
    # else:
    #     df_train_eh = df_train_eh.reindex(columns = df2_train_eh_before.columns, fill_value=0)

    # df_train_eh.to_csv('./data/df_train_eh.csv')
    
    

    # df_test_eh=pd.get_dummies(df.drop('MOVE_QTY', axis=1))
    #df_test_eh = df_test_eh.reindex(columns = df_test_eh.columns, fill_value=0)
   
    #np.asarray(x).astype('float32')        
           
    X_dropped = np.asarray(df_train_eh)

    Y_dropped = np.asarray(df[target_cols]) 
    return X_dropped,Y_dropped,df_train_eh
    

In [45]:
# # load the diabetes dataset
# df_train_orign = pd.read_csv('./data/TRCT_TrainingData_20210131.csv')
df_train_orign = pd.read_csv(trainDataFile)
 
# df_train_orign = df_train_orign.loc[df_train_orign['TOOLG_ID']==target_ToolGID]
#df_train_orign['MFG_DATE'] = pd.to_datetime(test['MFG_DATE'],format='%Y%m%d') 
df_train_orign['MFG_DATE'] = df_train_orign['MFG_DATE'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

MRATIO_df = df_train_orign

#刪除columns 值是空的()
MRATIO_df = MRATIO_df.dropna(axis=1, how='all')
 
# #刪除rows ,target值是空的()
MRATIO_df = MRATIO_df[MRATIO_df['MOVE_QTY'].notna()]
MRATIO_df = MRATIO_df[MRATIO_df['NO_HOLD_QTY'].notna()]

MRATIO_df['TOOLG_ID']=MRATIO_df['TOOLG_ID'].astype("category").cat.codes

MRATIO_df['TRCT']= MRATIO_df['NO_HOLD_QTY']/MRATIO_df['MOVE_QTY']

MRATIO_df.info()

MRATIO_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96956 entries, 0 to 128681
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   MFG_DATE         96956 non-null  datetime64[ns]
 1   TOOLG_ID         96956 non-null  int16         
 2   IS_HOLIDAY       96956 non-null  float64       
 3   M_NUM            96956 non-null  float64       
 4   UP_TIME          96956 non-null  float64       
 5   C_UP_TIME        96955 non-null  float64       
 6   LOT_SIZE         96956 non-null  float64       
 7   C_LOT_SIZE       96955 non-null  float64       
 8   EQP_UTIL         96956 non-null  float64       
 9   C_EQP_UTIL       96955 non-null  float64       
 10  U                96866 non-null  float64       
 11  PROCESS_TIME     96956 non-null  float64       
 12  WIP_QTY          96956 non-null  float64       
 13  NO_HOLD_QTY      96956 non-null  float64       
 14  MOVE_QTY         96956 non-null  floa

MFG_DATE            0
TOOLG_ID            0
IS_HOLIDAY          0
M_NUM               0
UP_TIME             0
C_UP_TIME           1
LOT_SIZE            0
C_LOT_SIZE          1
EQP_UTIL            0
C_EQP_UTIL          1
U                  90
PROCESS_TIME        0
WIP_QTY             0
NO_HOLD_QTY         0
MOVE_QTY            0
ARRIVAL_WIP_QTY     0
RUN_WIP_RATIO       0
C_TC                1
HOLD_RATE           0
ENG_LOT_RATE        0
HOT_LOT_RATE        0
QUE_LOT_RATE        0
TRCT                0
dtype: int64

In [46]:

# drop_cols=['MFG_DATE']
# MRATIO_df = MRATIO_df.drop(drop_cols, axis=1)


In [103]:
 
# np.where(np.isnan(x))
# np.nan_to_num(x)

 

# print(np.where(x >= np.finfo(np.float64).max))
# print(np.isnan(x.any()))
# print(np.where(y >= np.finfo(np.float64).max))
# print(np.isnan(y.any()))


# print(np.where(x_train >= np.finfo(np.float64).max))
# print(np.isnan(x_train.any()))
# print(np.where(y_train >= np.finfo(np.float64).max))
# print(np.isnan(y_train.any()))

x, y ,df_onehot= preHandleDat(MRATIO_df,True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
 

In [107]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [108]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train) 
# print("Features sorted by their score:")
# print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
#              reverse=True))

ipykernel_launcher:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


KeyboardInterrupt: 

In [100]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = df_onehot.columns[1:]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) RUN_WIP_RATIO                  0.530184
 2) NO_HOLD_QTY                    0.130048
 3) EQP_UTIL                       0.102521
 4) TOOLG_ID_280                   0.028065
 5) LOT_SIZE                       0.022783
 6) C_LOT_SIZE                     0.020713
 7) C_EQP_UTIL                     0.018538
 8) U                              0.016115
 9) C_UP_TIME                      0.013033
10) ENG_LOT_RATE                   0.010868
11) C_TC                           0.010819
12) TOOLG_ID_0                     0.010541
13) HOT_LOT_RATE                   0.010431
14) WIP_QTY                        0.009530
15) HOLD_RATE                      0.008330
16) UP_TIME                        0.006389
17) QUE_LOT_RATE                   0.006018
18) TOOLG_ID_147                   0.005876
19) M_NUM                          0.005760
20) PROCESS_TIME                   0.005175
21) TOOLG_ID_55                    0.004612
22) TOOLG_ID_75                    0.003566
23) TOOLG_ID_290                

IndexError: index 337 is out of bounds for axis 0 with size 337

In [55]:

x, y ,df_onehot= preHandleDat(MRATIO_df,True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
# feat_labels = df.columns[1:]

forest = RandomForestRegressor(n_estimators=10000, random_state=0, n_jobs=-1)
# forest.fit(x_train, y_train)
 

In [65]:
x, y ,df_onehot= preHandleDat(MRATIO_df,True)

In [56]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)
forest.fit(x_train, y_train)

ipykernel_launcher:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


KeyboardInterrupt: 

In [ ]:
res = pd.DataFrame({
    "features":var_names,
    "importance":random_forest.feature_importances_,
    })
    res = res.sort_values(['importance'],ascending=False)
    res['rank'] = np.arange(len(res))+1